# This section uses Magpie to calculate surface brightness profile

This is equivalently what the previous section does, but magpie is used to calculate surface brightness profile for each Ha Hb line maps and weights, averaged out over the polar angle phi, for each given pixel distance.

what does the package [magpie](https://github.com/knaidoo29/magpie/tree/master) do:

It transforms grid in Cartesian coordinates into (r,phi) polar coordinates, while preserving the size of the surface area of each pixel. Therefore one can calculate surface brightness of a certain ring area by summing over phi of a given radius range. This method is used in [Matharu 2023](https://iopscience.iop.org/article/10.3847/2041-8213/acd1db/pdf) and [Matharu 2024](https://arxiv.org/pdf/2404.17629).

Update: Now this script also calculates dust attenuation assuming [Calzetti et al. (2000)](https://ui.adsabs.harvard.edu/abs/2000ApJ...533..682C/abstract)

In [1]:
import  magpie              as     magpie

### calculate radial profile

In [2]:
import  numpy               as     np
from    astropy.table       import Table
from    astropy.io          import fits
from    astropy.cosmology   import Planck18
import  astropy.units       as     u
import  matplotlib.pyplot   as     plt
import  matplotlib.colors   as     colors  
from    matplotlib          import use
from    tqdm                import tqdm
from    concurrent.futures  import ThreadPoolExecutor, as_completed
import  sys, os
from    IPython.display     import clear_output



#this is just a handy little function to return the desired file path
#give one entry in the object list, return the desired file path
def file_name(obj,prefix,filetype='fits'):
    field = obj['subfield'].lower()
    id    = str(obj['ID']).zfill(5)
    return f"hlsp_clear_hst_wfc3_{field}-{id}_g102-g141_v4_{prefix}.{filetype}"

#this function
def find_data(name,hdu):
    for image in hdu:
        if name == image.name:
            return image

#save updataed or add hduimage to the extracted file
def save_update(image_to_save,extracted):
        for i,image in enumerate(extracted):
            if image.name == image_to_save.name:
                extracted[i] = image_to_save
                return extracted
        extracted.append(image_to_save)
        extracted.flush()
        return extracted

#this is just a small function to count error from catagory process
def errorcounting(results):
    number = 0
    for result in results:
        if 'error' in result:
            number +=1
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)

obj_lis = Table.read('obj_lis_selected.fits')

import magpie.montecarlo
#use the montecarlo module to remap the data to polar coordinates
def spatial_remap(map,pixel_length,r_bins=25):
    b2r = magpie.montecarlo.Box2Ring()
    b2r.setup_box(-25*pixel_length,25*pixel_length,50,
                -25*pixel_length,25*pixel_length,50)
    b2r.setup_polar_lin(0., 25*pixel_length, r_bins, 10, center=[0., 0.])
    b2r.get_weights() 
    return np.linspace(b2r.redges[0], b2r.redges[-1],r_bins), b2r.remap(map)

def K_lambda(line='Ha'):
    #calzetti_attenuation
    """
    Calculate the dust attenuation value k(lambda) from the Calzetti et al. (2000) attenuation curve.
    Parameters:
        wavelength_nm (float): Wavelength in nanometers (nm).
    """
    wavelength_um =  (0.6563 if line == 'Ha' else 0.4861)
    if 0.12 <= wavelength_um <= 0.63:
        # Formula for the UV to optical wavelength range
        k_lambda = 2.659 * (-1.857 + 1.040 / wavelength_um) + 4.05
    elif wavelength_um > 0.63:
        # Formula for the near-infrared wavelength range
        k_lambda = 2.659 * (-2.156 + 1.509 / wavelength_um - 0.198 / (wavelength_um ** 2) + 0.011 / (wavelength_um ** 3)) + 4.05
    return k_lambda


#this function will return the radial profile of a given object
#function: calls the magpie function spatial_remap to remap the data to polar coordinates
def radial_profile(obj,linemap,weight,seg,pixel_length):
        linemap = linemap.data; weight = weight.data; seg = seg.data

        #convert the seg map to a binary map, for further conversion to weight map
        seg = np.where(seg==obj['ID'],1,0)
        seg = spatial_remap(seg,pixel_length)[1]
        seg_weight = seg/np.sum(seg,axis=0)

        r, linemap_r,   = spatial_remap(np.where(linemap>0,linemap,0),pixel_length)
        linemap_r_wht = spatial_remap(np.where(linemap>0,weight,0),pixel_length)[1]


        map_r   = np.average(linemap_r,weights=seg_weight,axis=0)
        map_r_err = 1/np.sum(linemap_r_wht*seg,axis=0)**0.5
        map_r_std = np.average((linemap_r-map_r)**2,weights = seg_weight,axis=0)**0.5
        map_r_err = (map_r_err**2 + map_r_std**2)**0.5
        return r, map_r, map_r_err 

#this function will generate the radial table for a given object
def gen_radial_table(obj,
                     LINE_HA='LINE_HA',      LINE_HB='LINE_HB_CONV',
                     LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV'):
    #try:
    path = f"data_extracted/{file_name(obj,prefix='extracted')}"
    with fits.open(path,mode='update') as hdu:
        #extract the radial profile surface brightness
        r, ha_r, ha_r_err = radial_profile(obj,
                                        linemap      = find_data(LINE_HA,hdu),
                                        weight       = find_data(LINEWHT_HA,hdu),
                                        seg          = find_data('SEG',hdu),
                                        pixel_length = obj['pixel_length'])
        
        r, hb_r, hb_r_err = radial_profile(obj,
                                        linemap      = find_data(LINE_HB,hdu),
                                        weight       = find_data(LINEWHT_HB,hdu),
                                        seg          = find_data('SEG',hdu),
                                        pixel_length = obj['pixel_length'])

        #calculate the balmer decrement
        balmer_r     = ha_r/hb_r
        balmer_r_err = ((ha_r_err/hb_r)**2 + (hb_r_err**2 * (ha_r/hb_r**2)**2))**0.5
        
        #now calculate the extinction
        #color excess
        E_ba = 2.5*np.log10(balmer_r/2.86)
        #attenutation
        A_ba = E_ba/(K_lambda('Hb')-K_lambda('Ha')) * K_lambda('Ha')

        #columns for the radial table
        cols = [
            fits.Column(name='DISTANCE [kpc]',                       format='E', array=r),
            fits.Column(name='Ha_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=ha_r),
            fits.Column(name='Ha_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=ha_r_err),
            fits.Column(name='Hb_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=hb_r),
            fits.Column(name='Hb_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=hb_r_err),
            fits.Column(name='BALMER_DECREM',                        format='E', array=balmer_r),
            fits.Column(name='BALMER_DECREM_ERR',                    format='E',array=balmer_r_err),
            fits.Column(name='E_BV',                                  format='E', array=E_ba),
            fits.Column(name='A_Ha',                                  format='E', array=A_ba)
        ]
        
        #choose the right name for saving the radial table
        if 'CONV' in LINE_HB:
            name_addon = '_CONV'
        else:
            name_addon = ''
        if 'BG' not in LINE_HA:
            name = f'RAD_PROFILE{name_addon}'
            new_table = fits.BinTableHDU.from_columns(cols, name=name)
        else:
            name = f'RAD_PROFILE{name_addon}_BG'
            new_table = fits.BinTableHDU.from_columns(cols, name=name)

        #save or update table
        save_update(new_table,hdu)
        hdu.flush()
        return f"{obj['subfield']}-{obj['ID']} processed"
    #except Exception as e:
    #        return f"! {obj['subfield']}-{obj['ID']} failed, error{e}"


def cat_process(obj_lis,
                LINE_HA,   LINE_HB,
                LINEWHT_HA, LINEWHT_HB,
                max_threads=1):
        print(f'start process,{LINE_HA},{LINE_HB}')
        results = []
        if max_threads > 1 :
            with ThreadPoolExecutor(max_threads) as executor:
                futures = {executor.submit(
                    gen_radial_table,
                    obj,LINE_HA,LINE_HB,LINEWHT_HA,LINEWHT_HB
                                            ): obj for obj in obj_lis}
                for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                    results.append(future.result())
            return results
        else:
            for obj in obj_lis:
                results.append(gen_radial_table(obj,LINE_HA,LINE_HB,LINEWHT_HA,LINEWHT_HB))
            return results


def main():
    obj_lis = Table.read('obj_lis_selected.fits')
    
    results1 = cat_process(obj_lis,
                        LINE_HA='LINE_HA',LINE_HB='LINE_HB',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB',max_threads=6)
    errorcounting(results1)

    results3 = cat_process(obj_lis,
                        LINE_HA='LINE_HA',LINE_HB='LINE_HB_CONV',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV',max_threads=6)
    errorcounting(results3)

'''
    results2 = cat_process(obj_lis,
                        LINE_HA='LINE_HA_BG',LINE_HB='LINE_HB_BG',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB',max_threads=6)
    errorcounting(results2)
    
    results4 = cat_process(obj_lis,
                        LINE_HA='LINE_HA_BG',LINE_HB='LINE_HB_CONV_BG',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV',max_threads=6)
    errorcounting(results4)
'''
if __name__ == '__main__':
    main()


start process,LINE_HA,LINE_HB


Processing:   0%|          | 0/158 [00:00<?, ?it/s]/tmp/ipykernel_6043/534480588.py:85: RuntimeWarning: invalid value encountered in divide
  seg_weight = seg/np.sum(seg,axis=0)
/tmp/ipykernel_6043/534480588.py:92: RuntimeWarning: divide by zero encountered in divide
  map_r_err = 1/np.sum(linemap_r_wht*seg,axis=0)**0.5
/tmp/ipykernel_6043/534480588.py:123: RuntimeWarning: divide by zero encountered in log10
  E_ba = 2.5*np.log10(balmer_r/2.86)
Processing:   5%|▌         | 8/158 [00:23<07:18,  2.92s/it]/tmp/ipykernel_6043/534480588.py:118: RuntimeWarning: divide by zero encountered in divide
  balmer_r     = ha_r/hb_r
/tmp/ipykernel_6043/534480588.py:119: RuntimeWarning: divide by zero encountered in divide
  balmer_r_err = ((ha_r_err/hb_r)**2 + (hb_r_err**2 * (ha_r/hb_r**2)**2))**0.5
Processing: 100%|██████████| 158/158 [05:24<00:00,  2.05s/it]


total number of obj processed: 158
number of failed obj 0
start process,LINE_HA,LINE_HB_CONV


Processing: 100%|██████████| 158/158 [06:34<00:00,  2.50s/it]

total number of obj processed: 158
number of failed obj 0


### plot radial profiles

In [3]:
%matplotlib inline
def plot_balmer_decrem(obj, plot, plot_var, crop_size=30):
    try:
        path = f"data_extracted/{file_name(obj, prefix='extracted')}"
        with fits.open(path) as hdu:
            center_x, center_y = np.array(hdu[3].data.shape) // 2
            half_crop = crop_size // 2

            fig, axes = plt.subplots(3, 2, figsize=(12, 15))
            axes = axes.flatten()
            
            for i, name in enumerate(['DSCI', 'LINE_HA', 'LINE_HB_CONV']):
                image = find_data(name, hdu)
                data = image.data[
                    center_y - half_crop: center_y + half_crop,
                    center_x - half_crop: center_x + half_crop]
                ax = axes[i]
                im = ax.imshow(data, norm=colors.Normalize(vmin=0), origin='lower', cmap='plasma_r')
                ax.plot([3, 7], [4, 4])
                ax.text(5, 5, f"{round(obj['pixel_length'] * 4, 2)} kpc")
                ax.set_title(f"{image.name}_{obj['subfield']}_{obj['ID']}_{obj['tag']}_{obj['manual_select']}")
                fig.colorbar(im, ax=ax)

            name = '2D_BALMER'
            image = find_data(name, hdu)
            data = image.data[
                center_y - half_crop: center_y + half_crop,
                center_x - half_crop: center_x + half_crop]
            ax = axes[3]
            im = ax.imshow(data, norm=colors.Normalize(vmin=0, vmax=12), origin='lower', cmap='plasma_r')
            ax.plot([3, 7], [4, 4])
            ax.text(5, 5, f"{round(obj['pixel_length'] * 4, 2)} kpc")
            ax.set_title(f"{image.name}_{obj['subfield']}_{obj['ID']}_{obj['tag']}_{obj['manual_select']}")
            fig.colorbar(im, ax=ax)

            r, ha_r, ha_r_err, hb_r, hb_r_err, balmer_r, balmer_r_err, E_BV, A_Ha = np.vstack(find_data(plot, hdu).data).transpose()
            r, ha_r_var, ha_r_err_var, hb_r_var, hb_r_err_var, balmer_r_var, balmer_r_err_var, E_BV, A_Ha = np.vstack(find_data(plot_var, hdu).data).transpose()

            ax = axes[4]
            ax.errorbar(r, ha_r, yerr=ha_r_err, fmt='bo:', label=f'Ha_{plot}')
            ax.errorbar(r, hb_r, yerr=hb_r_err, fmt='go:', label=f'Hb_{plot}')
            ax.errorbar(r, ha_r_var, yerr=ha_r_err_var, fmt='bo:', label=f'Ha_{plot_var}', alpha=0.4)
            ax.errorbar(r, hb_r_var, yerr=hb_r_err_var, fmt='go:', label=f'Hb_{plot_var}', alpha=0.4)
            ax.set_xlabel('distance [kpc]')
            ax.set_ylabel('flux [1e-7 erg/s/cm2]')
            ax.set_yscale('log')
            ax.grid()
            ax.legend()

            ax = axes[5]
            ax.errorbar(r, balmer_r, yerr=balmer_r_err, fmt='ro:', label=f'balmer_decrem_{plot}')
            ax.errorbar(r, balmer_r_var, yerr=balmer_r_err_var, fmt='bo:', label=f'balmer_decrem_{plot_var}', alpha=0.4)
            ax.set_xlabel('distance [kpc]')
            ax.set_ylabel('Ha/Hb')
            ax.set_xlim(0, 8)
            ax.set_ylim(-5, 15)
            ax.grid()
            ax.legend()

            ax2 = ax.twinx()
            ax2.errorbar(r, A_Ha, yerr=0, fmt='go:', label='A_Ha', alpha=0.4)
            ax2.set_ylabel('E_BV, A_Ha')
            ax2.legend()


            save_path = f"radial_balmer_decrem/{plot}_vs_{plot_var}"
            save_path_sn_10 = f"sn_10/radial_balmer_decrem/{plot}_vs_{plot_var}"
            os.makedirs(save_path, exist_ok=True)
            os.makedirs(save_path_sn_10, exist_ok=True)

            plt.savefig(f"{save_path}/{obj['subfield']}-{obj['ID']}.png")
            if obj['sn_hb'] > 10:
                plt.savefig(f"{save_path_sn_10}/{obj['subfield']}-{obj['ID']}.png")
            plt.close('all')

            return f"{obj['subfield']}-{obj['ID']} saved"
    except Exception as e:
        return f"! {obj['subfield']}-{obj['ID']} failed, error: {e}"
        
    #except Exception as e:
    #        return f"! {obj['subfield']}-{obj['ID']} failed, error{e}"


def cat_process(obj_lis,plot='RAD_PROFILE',plot_var='RAD_PROFILE_BG',max_threads=1):
        print(f'\n start plot process{plot,plot_var}')
        results = []
        if max_threads>1:
            with ThreadPoolExecutor(max_threads) as executor:
                futures = {executor.submit(plot_balmer_decrem,
                                            obj,plot=plot,plot_var=plot_var
                                            ):obj for obj in obj_lis}
                for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                    results.append(future.result())
            return results
        else:
            for obj in tqdm(obj_lis):
                results.append(plot_balmer_decrem(obj,plot,plot_var))
            return results


def main():
    use('Agg')
    obj_lis = Table.read('obj_lis_selected.fits')[:30]
    
    results1 = cat_process(obj_lis,
                        plot='RAD_PROFILE',plot_var='RAD_PROFILE_CONV',max_threads=1)
    errorcounting(results1)
    '''
    results2 = cat_process(obj_lis,
                        plot='RAD_PROFILE',plot_var='RAD_PROFILE_BG',max_threads=1)
    errorcounting(results2)

    results3 = cat_process(obj_lis,
                        plot='RAD_PROFILE_BG',plot_var='RAD_PROFILE_CONV_BG',max_threads=1)
    errorcounting(results3)
    '''
if __name__ == '__main__':
    main()


 start plot process('RAD_PROFILE', 'RAD_PROFILE_CONV')


 37%|███▋      | 11/30 [00:10<00:17,  1.10it/s]/home/yang/anaconda3/envs/magpie/lib/python3.9/site-packages/matplotlib/axes/_axes.py:3749: RuntimeWarning: invalid value encountered in add
  low, high = dep + np.vstack([-(1 - lolims), 1 - uplims]) * err
100%|██████████| 30/30 [00:29<00:00,  1.01it/s]

total number of obj processed: 30
number of failed obj 0


In [3]:
plot_balmer_decrem(obj_lis[2],plot='RAD_PROFILE',plot_var='RAD_PROFILE_CONV')


'GN7-11839saved'